# Sequence Analysis and Functional Prediction Pipeline
This notebook is enterily run in colab

## 1. Introduction
This notebook analyzes the functional and sequence relationships between newly identified bacteria and known corrosion-influencing microorganisms. The analysis builds upon previous findings where:
- Statistical significance was established between the selected bacteria and corrosion risk (Notebook 3)
- Literature validation confirmed corrosion influence for many bacteria (Notebook 4)
- Evolutionary relationships were mapped through phylogenetic analysis (Notebook 5)

The study focuses on bacteria from operational heating and cooling water systems, primarily in Germany. Using 16S rRNA data (bootstrap-validated from Notebook 5), this analysis employs PICRUSt2 to predict metabolic functions and compare functional profiles between different bacterial groups.

### Analysis Approaches
We implement two classification strategies:

1. Simple Classification:
   - Known corrosion-causing bacteria (usual_taxa)
   - Other bacteria (combining checked_taxa and core_taxa)

2. Detailed Classification:
   - Known corrosion-causing bacteria (usual_taxa)
   - Pure checked bacteria (exclusive to checked_taxa)
   - Pure core bacteria (exclusive to core_taxa)
   - Checked-core bacteria (overlap between checked and core taxa)

This detailed approach allows for more nuanced analysis of functional profiles and better understanding of potential corrosion mechanisms across different bacterial groups.

### Analysis Goals:
- Predict metabolic functions from 16S sequences
- Focus on corrosion-relevant pathways (sulfur/iron metabolism)
- Compare functional profiles between known corrosion-causing bacteria and newly identified candidates
- Validate whether statistical correlations reflect genuine metabolic capabilities associated with corrosion processes

### Directory Structure:
 Following is the structure of the notebook data named data_picrus  
data_tree  
 ├── sequences/  
 │   ├── known.fasta : sequences of known corrosion-causing bacteria  
 │   ├── candidate.fasta : sequences of potential new corrosion-causing bacteria  
 |   └── other files  
 data_picrus  
 └── picrust_results/  
      ├── known_bacteria/  
      |               ├── EC_predictions/       : enzyme predictions  
      |               ├── pathway_predictions/  : metabolic pathway abundance  
      |               ├── KO_predictions/       : KEGG ortholog predictions  
      |               └── other_picrust_files/  
      ├── candidate_bacteria/  
      |               ├── EC_predictions/       : enzyme predictions  
      |               ├── pathway_predictions/  : metabolic pathway abundance  
      |               ├── KO_predictions/       : KEGG ortholog predictions  
      |               └── other_picrust_files/  : final comparison summary
      ├── core_bacteria/
      |               ├── EC_predictions/       : enzyme predictions  
      |               ├── pathway_predictions/  : metabolic pathway abundance  
      |               ├── KO_predictions/       : KEGG ortholog predictions  
      |               └── other_picrust_files/  
      │      
      └── functional_comparison.xlsx  
Picrust2 works using its reference database that was installed with the package /home/beatriz/miniconda3/envs/picrust2/lib/python3.9/site-packages/picrust2/default_files/prokaryotic/pro_ref

# 2. Loading and Preparing the Data

## 2.1 Imports, Directories, Loading and preparing the Abundance DataFrame
The abundance DataFrame (Integrated) was carefully prepared to meet PICRUSt2 input requirements, including proper taxonomic level organization and removal of unnamed or missing data. The sequence data is sourced directly from aligned_sequences_integrated.fasta, which contains the phylogenetically aligned sequences generated in notebook 5. This integration ensures consistency between abundance data and sequence information.

Importing QIIME AND PICRUST IN COLAB

In [6]:
'''# Install miniconda and initialize
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local/miniconda3
!conda config --add channels defaults
!conda config --add channels bioconda
!conda config --add channels conda-forge'''

'# Install miniconda and initialize\n!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh\n!bash Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local/miniconda3\n!conda config --add channels defaults\n!conda config --add channels bioconda\n!conda config --add channels conda-forge'

In [7]:
'''import sys
sys.path.append('/usr/local/miniconda3/lib/python3.7/site-packages/')

# Create environment with QIIME2-2020.8 (stable version known to work with PICRUSt2)
!conda create -n qiime2-2020.8 python=3.7 -y
!conda activate qiime2-2020.8

# Install QIIME2
!wget https://data.qiime2.org/distro/core/qiime2-2020.8-py36-linux-conda.yml
!conda env update -n qiime2-2020.8 --file qiime2-2020.8-py36-linux-conda.yml

# Install PICRUSt2 and its dependencies
!conda install -c bioconda -c conda-forge picrust2=2.4.1 -y'''

"import sys\nsys.path.append('/usr/local/miniconda3/lib/python3.7/site-packages/')\n\n# Create environment with QIIME2-2020.8 (stable version known to work with PICRUSt2)\n!conda create -n qiime2-2020.8 python=3.7 -y\n!conda activate qiime2-2020.8\n\n# Install QIIME2\n!wget https://data.qiime2.org/distro/core/qiime2-2020.8-py36-linux-conda.yml\n!conda env update -n qiime2-2020.8 --file qiime2-2020.8-py36-linux-conda.yml\n\n# Install PICRUSt2 and its dependencies\n!conda install -c bioconda -c conda-forge picrust2=2.4.1 -y"

In [8]:


'''# Verify installations
!conda list | grep qiime2
!conda list | grep picrust2

# Function to check if the installations were successful
def check_installations():
    try:
        import qiime2
        print("QIIME2 installation successful")
        print(f"QIIME2 version: {qiime2.__version__}")
    except ImportError:
        print("QIIME2 installation failed")

    try:
        !which picrust2_pipeline.py
        print("PICRUSt2 installation successful")
    except:
        print("PICRUSt2 installation failed")

check_installations()'''


'# Verify installations\n!conda list | grep qiime2\n!conda list | grep picrust2\n\n# Function to check if the installations were successful\ndef check_installations():\n    try:\n        import qiime2\n        print("QIIME2 installation successful")\n        print(f"QIIME2 version: {qiime2.__version__}")\n    except ImportError:\n        print("QIIME2 installation failed")\n\n    try:\n        !which picrust2_pipeline.py\n        print("PICRUSt2 installation successful")\n    except:\n        print("PICRUSt2 installation failed")\n\ncheck_installations()'

In [9]:
conda list dendropy

# packages in environment at /home/beatriz/miniconda3/envs/picrust2_new:
#
# Name                    Version                   Build  Channel

Note: you may need to restart the kernel to use updated packages.


In [10]:
# Download test data
'''!wget http://kronos.pharmacology.dal.ca/public_files/picrust/picrust2_tutorial_files/mammal_biom.qza
!wget http://kronos.pharmacology.dal.ca/public_files/picrust/picrust2_tutorial_files/mammal_seqs.qza'''

'!wget http://kronos.pharmacology.dal.ca/public_files/picrust/picrust2_tutorial_files/mammal_biom.qza\n!wget http://kronos.pharmacology.dal.ca/public_files/picrust/picrust2_tutorial_files/mammal_seqs.qza'

In [11]:
'''#Example
# Run PICRUSt2 through QIIME2
qiime picrust2 full-pipeline \
    --i-table mammal_biom.qza \
    --i-seq mammal_seqs.qza \
    --output-dir q2-picrust2_output \
    --p-placement-tool sepp \
    --p-threads 1 \
    --p-hsp-method pic \
    --p-max-nsti 2 \
    --verbose

# Function to analyze the output
def check_output():
    import os
    output_files = os.listdir('q2-picrust2_output')
    print("Generated output files:")
    for file in output_files:
        print(f"- {file}")

check_output()

Instructions for using this notebook:

1. Create a new Colab notebook
2. Copy this entire code into the notebook
3. Run the cells in order
4. The installation may take 5-10 minutes
5. After installation, you can use QIIME2 and PICRUSt2 commands

Common troubleshooting:
- If you get memory errors, try restarting the runtime
- Make sure to run cells in order
- Check that all installations completed successfully
- If you get path errors, make sure conda environment is activated

To use your own data:
1. Upload your feature table (.qza format)
2. Upload your sequence file (.qza format)
3. Modify the PICRUSt2 command with your file names'''

'#Example\n# Run PICRUSt2 through QIIME2\nqiime picrust2 full-pipeline     --i-table mammal_biom.qza     --i-seq mammal_seqs.qza     --output-dir q2-picrust2_output     --p-placement-tool sepp     --p-threads 1     --p-hsp-method pic     --p-max-nsti 2     --verbose\n\n# Function to analyze the output\ndef check_output():\n    import os\n    output_files = os.listdir(\'q2-picrust2_output\')\n    print("Generated output files:")\n    for file in output_files:\n        print(f"- {file}")\n\ncheck_output()\n\nInstructions for using this notebook:\n\n1. Create a new Colab notebook\n2. Copy this entire code into the notebook\n3. Run the cells in order\n4. The installation may take 5-10 minutes\n5. After installation, you can use QIIME2 and PICRUSt2 commands\n\nCommon troubleshooting:\n- If you get memory errors, try restarting the runtime\n- Make sure to run cells in order\n- Check that all installations completed successfully\n- If you get path errors, make sure conda environment is activa

In [12]:
'''from google.colab import drive
drive.mount('/content/drive')

#change the path
os.chdir('/content/drive/My Drive/MIC/picrust')'''

"from google.colab import drive\ndrive.mount('/content/drive')\n\n#change the path\nos.chdir('/content/drive/My Drive/MIC/picrust')"

In [13]:
'''# Imports for colab
import condacolab'''

'# Imports for colab\nimport condacolab'

In [1]:
# Standard library imports
import os
import ast
import subprocess
import logging
import shutil
from io import StringIO
from pathlib import Path
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# Data processing imports
import pandas as pd
import numpy as np
import openpyxl
import matplotlib.pyplot as plt

# BIOM handling
from biom import Table
from biom.util import biom_open
from biom import load_table

In [10]:
# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Directory Structure Definitions
SIMPLE_BASE = {
    'known': 'simple_known_mic',
    'other': 'simple_candidate_mic'
}

DETAILED_BASE = {
    'known': 'detailed_known_mic',
    'pure_checked': 'detailed_pure_checked_mic',
    'pure_core': 'detailed_pure_core_mic',
    'checked_core': 'detailed_checked_core_mic'
}

SUBDIRS = [
    'EC_predictions',
    'pathway_predictions',
    'KO_predictions',
    'other_picrust_files'
]

# Base Paths
base_dir = Path("/home/beatriz/MIC/2_Micro/data_picrust")
# Create output directory if it doesn't exist
base_dir.mkdir(parents=True, exist_ok=True)
abundance_excel= Path("/home/beatriz/MIC/2_Micro/data_Ref/merged_to_sequence.xlsx")

fasta_file = Path("/home/beatriz/MIC/2_Micro/data_picrust/sequences_for_picrust.fasta")
biom_table = Path("/home/beatriz/MIC/2_Micro/data_picrust/abundance_otus_85.biom")
results_file = base_dir / "functional_comparison.xlsx"
output_dir = base_dir / "picrust2_out"  # Separate output directory

The fasta file was created on Notebook 5_sequences_qiime using an alternative approach with the greenes database, the biom_table also was created previously. 

In [17]:
# Integrated taxa from origin genus as headers with levels 6 for the genera, 7 for the GID, muss be cleaned
Integrated_T = pd.read_excel(abundance_excel, sheet_name='core_check_usual_taxa', header=[0,1,2,3,4,5,6,7], engine ='openpyxl')
# Drop first row (index 0) and first column in one chain
Integrated_T = Integrated_T.drop(index=0).drop(Integrated_T.columns[0], axis=1)
# Remove 'Unnamed' level names
Integrated_T.columns = Integrated_T.columns.map(lambda x: tuple('' if 'Unnamed' in str(level) else level for level in x))
# If the dataframe has Nan in sites it will replace it with Source
Integrated_T['Sites'] = Integrated_T['Sites'].fillna('Source')
# Fill the other index with nothing
Integrated_T =  Integrated_T.fillna(' ')
Integrated_T= Integrated_T.set_index("Sites")
# sources are  array([' ', 'chk-core', 'chk', 'chk-core-us', 'chk-us', 'core-us', 'core', 'us'], dtype=object)

In [2]:
biom_table = load_table("/home/beatriz/MIC/2_Micro/data_picrust/abundance_otus_85.biom")

In [11]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

# Read and modify sequences
new_records = []
for record in SeqIO.parse("/home/beatriz/MIC/2_Micro/data_qiime/results_match_gg/final_sequences_gg.fasta", "fasta"):
    # Get just the OTU number and ignore genus name
    otu_id = record.description.split()[1]  # Takes second element (the OTU number)
    
    # Create new record with only OTU as ID
    new_record = SeqRecord(
        record.seq,
        id=otu_id,
        description=""  # Empty description to keep only ID
    )
    new_records.append(new_record)

# Write modified FASTA
SeqIO.write(new_records, "/home/beatriz/MIC/2_Micro/data_picrust/sequences_for_picrust.fasta", "fasta")

85

In [13]:
def run_picrust2_pipeline():
    """Run the main PICRUSt2 pipeline"""
    if output_dir.exists():
        shutil.rmtree(output_dir)
        logging.info(f"Removed existing directory: {output_dir}")
    
    # Modified command with correct parameters
    cmd = [
        'picrust2_pipeline.py',
        '-s', str(fasta_file),
        '-i', str(biom_table),
        '-o', str(output_dir),
        '--threads', '1',  # Changed from --processes
        '-a', 'hmmalign',  # Specify alignment method
        '-m', 'mp',        # Use maximum parsimony method
        '--verbose'
    ]
    
    try:
        logging.info("Starting PICRUSt2 pipeline...")
        subprocess.run(cmd, check=True)
        logging.info("PICRUSt2 pipeline completed successfully")
        return True
    except subprocess.CalledProcessError as e:
        logging.error(f"Error running PICRUSt2: {e}")
        return False

In [14]:
test =run_picrust2_pipeline()

2025-01-28 08:45:23,793 - INFO - Removed existing directory: /home/beatriz/MIC/2_Micro/data_picrust/picrust2_out
2025-01-28 08:45:23,812 - INFO - Starting PICRUSt2 pipeline...
2025-01-28 08:45:27,764 - ERROR - Error running PICRUSt2: Command '['picrust2_pipeline.py', '-s', '/home/beatriz/MIC/2_Micro/data_picrust/sequences_for_picrust.fasta', '-i', '/home/beatriz/MIC/2_Micro/data_picrust/abundance_otus_85.biom', '-o', '/home/beatriz/MIC/2_Micro/data_picrust/picrust2_out', '--threads', '1', '-a', 'hmmalign', '-m', 'mp', '--verbose']' returned non-zero exit status 1.


In [24]:
import logging
import subprocess
import shutil
from pathlib import Path

# Set up logging
logging.basicConfig(level=logging.INFO,
                   format='%(asctime)s - %(levelname)s - %(message)s')

# Directory Structure Definitions
SIMPLE_BASE = {
    'known': 'simple_known_mic',
    'other': 'simple_candidate_mic'
}

DETAILED_BASE = {
    'known': 'detailed_known_mic',
    'pure_checked': 'detailed_pure_checked_mic',
    'pure_core': 'detailed_pure_core_mic',
    'checked_core': 'detailed_checked_core_mic'
}

SUBDIRS = [
    'EC_predictions',
    'pathway_predictions', 
    'KO_predictions',
    'other_picrust_files'
]

# Base Paths
base_dir = Path("/home/beatriz/MIC/2_Micro/data_picrust")
fasta_file = Path("/home/beatriz/MIC/2_Micro/data_picrust/sequences_for_picrust.fasta")
biom_table = Path("/home/beatriz/MIC/2_Micro/data_picrust/abundance_otus_85.biom")
results_file = base_dir / "functional_comparison.xlsx"
output_dir = base_dir / "picrust2_out"  # Separate output directory

def create_directory_structure():
    """Create the basic directory structure for PICRUSt2 analysis"""
    try:
        # Create base directory
        base_dir.mkdir(parents=True, exist_ok=True)
        
        # Create simple classification directories
        for dir_name in SIMPLE_BASE.values():
            for subdir in SUBDIRS:
                (base_dir / dir_name / subdir).mkdir(parents=True, exist_ok=True)
                
        logging.info("Directory structure created successfully")
        return True
        
    except Exception as e:
        logging.error(f"Error creating directory structure: {str(e)}")
        return False

def verify_input_files():
    """Verify that input files exist and are readable"""
    missing_files = []
    
    if not fasta_file.exists():
        missing_files.append(str(fasta_file))
    if not biom_table.exists():
        missing_files.append(str(biom_table))
        
    if missing_files:
        logging.error(f"Missing input files: {', '.join(missing_files)}")
        return False
    
    logging.info("All input files found")
    return True

def run_picrust2_pipeline():
    """Run the main PICRUSt2 pipeline"""
    # Remove only picrust2_out directory if it exists
    if output_dir.exists():
        shutil.rmtree(output_dir)
        logging.info(f"Removed existing directory: {output_dir}")
    
    cmd = [
        'picrust2_pipeline.py',
        '-s', str(fasta_file),
        '-i', str(biom_table),
        '-o', str(output_dir),
        '--threads', '1',
        '--verbose'
    ]
    
    try:
        logging.info("Starting PICRUSt2 pipeline...")
        subprocess.run(cmd, check=True)
        logging.info("PICRUSt2 pipeline completed successfully")
        return True
    except subprocess.CalledProcessError as e:
        logging.error(f"Error running PICRUSt2: {e}")
        return False

# Main execution
if __name__ == "__main__":
    if create_directory_structure() and verify_input_files():
        run_picrust2_pipeline()
    else:
        logging.error("Setup failed. Please check the errors above.")

2025-01-27 21:56:33,452 - INFO - Directory structure created successfully
2025-01-27 21:56:33,579 - INFO - All input files found
2025-01-27 21:56:33,596 - INFO - Removed existing directory: /home/beatriz/MIC/2_Micro/data_picrust/picrust2_out
2025-01-27 21:56:33,597 - INFO - Starting PICRUSt2 pipeline...
2025-01-27 21:56:35,383 - ERROR - Error running PICRUSt2: Command '['picrust2_pipeline.py', '-s', '/home/beatriz/MIC/2_Micro/data_picrust/sequences_for_picrust.fasta', '-i', '/home/beatriz/MIC/2_Micro/data_picrust/abundance_otus_85.biom', '-o', '/home/beatriz/MIC/2_Micro/data_picrust/picrust2_out', '--threads', '1', '--verbose']' returned non-zero exit status 1.


I have now download your adviced package and running the pipeline gives this error 

In [22]:
def process_integrated_data(df):
    """
    Process the integrated DataFrame to create a new DataFrame with clear column names
    and preserve all values including source information.

    Parameters:
    df (pandas.DataFrame): Input DataFrame with MultiIndex index and site columns

    Returns:
    pandas.DataFrame: Processed DataFrame with clear structure
    """

    # Extract genera and GIDs from the index MultiIndex
    genera = df.index.get_level_values(6)[1:]  # Skip first row
    gids = pd.to_numeric(df.index.get_level_values(7)[1:], errors='coerce')

    # Create a new DataFrame with the extracted information
    result_df = pd.DataFrame({
        'Genus': genera,
        'GID': gids
    })

    # Add the site values from the original DataFrame
    for col in df.columns:
        result_df[col] = df.iloc[1:][col].values

    # Clean up the DataFrame
    result_df['GID'] = pd.to_numeric(result_df['GID'], errors='coerce')
    result_df = result_df.dropna(subset=['GID'])
    result_df['GID'] = result_df['GID'].astype(int)

    return result_df

def get_taxa_groups(df):
    """
    Separate the processed DataFrame into different taxa groups based on Source column

    Parameters:
    df (pandas.DataFrame): Processed DataFrame from process_integrated_data()

    Returns:
    dict: Dictionary containing DataFrames for different taxa groups
    """
    # Split the data into groups based on 'Source' column patterns

    # Known corrosion bacteria (any pattern with 'us')
    known_bacteria = df[df['Source'].str.contains('us', case=False, na=False)]

    # Pure checked bacteria (only 'chk' without 'core' or 'us')
    pure_checked = df[
        df['Source'].str.contains('chk', case=False, na=False) &
        ~df['Source'].str.contains('core|us', case=False, na=False)
    ]

    # Pure core bacteria (only 'core' without 'chk' or 'us')
    pure_core = df[
        df['Source'].str.contains('core', case=False, na=False) &
        ~df['Source'].str.contains('chk|us', case=False, na=False)
    ]

    # Checked-core bacteria (contains both 'core' and 'chk' but no 'us')
    checked_core = df[
        df['Source'].str.contains('chk.*core|core.*chk', case=False, na=False) &
        ~df['Source'].str.contains('us', case=False, na=False)
    ]

    # Create groups dictionary
    taxa_groups = {
        'known_bacteria': known_bacteria,
        'pure_checked': pure_checked,
        'pure_core': pure_core,
        'checked_core': checked_core
    }

    # Print summary statistics
    print("\nDetailed Classification Results:")
    print(f"Known corrosion bacteria: {len(known_bacteria)}")
    print(f"Pure checked bacteria: {len(pure_checked)}")
    print(f"Pure core bacteria: {len(pure_core)}")
    print(f"Checked-core bacteria: {len(checked_core)}")

    # Verify total matches expected
    total_classified = len(known_bacteria) + len(pure_checked) + len(pure_core) + len(checked_core)
    print(f"\nTotal classified taxa: {total_classified}")
    print(f"Total in dataset: {len(df)}")

    return taxa_groups

# Usage example:
Integrated = process_integrated_data(pre_Integrated)

# Get the groups
taxa_groups = get_taxa_groups(Integrated)

# Access individual groups -
known_bacteria = taxa_groups['known_bacteria']
pure_core = taxa_groups['pure_core']
pure_checked = taxa_groups['pure_checked']
checked_core = taxa_groups['checked_core']

NameError: name 'pre_Integrated' is not defined

## 2.6. Classifying Bacteria by their Source DataFrame
Two distinct classification approaches are implemented to categorize bacteria. The simple approach (get_bacteria_sources_simple) divides bacteria into known corrosion-causers (usual_taxa) and candidates (all others). The detailed approach (get_bacteria_sources_detailed) provides finer categorization by separating bacteria into known corrosion-causers, pure checked taxa, pure core taxa, and those present in both checked and core datasets. Please notice that this function uses df Integrated for source clasification and no abundance.biom which will be used for the picrust2 pipeline.

In [ ]:
def get_bacteria_sources_simple(Integrated_df):
    """
    Simple classification:
    1. Known (anything with 'us')
    2. All others (combined chk, core, chk-core)
    """
    # Get genera and gids from column levels 6 and 7
    genera = Integrated_df["Genus"]
    gids = Integrated_df["GID"]
    # Look for Source in the data, not index
    sources = Integrated_df['Source'] if 'Source' in Integrated_df.columns else None

    known_bacteria = {}     # usual_taxa
    other_bacteria = {}     # everything else

    sources_found = set()
    source ={}
    patterns = ['us', 'core-us', 'chk-us', 'chk-core-us']

    for i, (genus, gid) in enumerate (zip(genera, gids)):
        if source is not None:  # Check if source exists for this genus
            source = str(sources.iloc[i]).strip().lower()
            sources_found.add(source)

            if source in patterns:
                known_bacteria[genus] = int(gid) if str(gid).isdigit() else gid
            else:
                other_bacteria[genus] = int(gid) if str(gid).isdigit() else gid

    print("\nSimple Classification Results:")
    print(f"Known corrosion bacteria: {len(known_bacteria)}")
    print(f"Other bacteria: {len(other_bacteria)}")
    print("\nSources found:", sources_found)

    return {
        'known_bacteria': known_bacteria,
        'other_bacteria': other_bacteria
    }

def get_bacteria_sources_detailed(Integrated_df):
    """
    Detailed classification with all possible combinations:
    1. Known (usual_taxa)
    2. Pure checked (only 'chk')
    3. Pure core (only 'core')
    4. Checked-core (overlap 'chk-core')
    """
    # Get genera and gids from column levels 6 and 7
    genera = Integrated_df.index.get_level_values(6)[1:]
    gids = Integrated_df.index.get_level_values(7)[1:]

    sources = Integrated_df['Source'] if 'Source' in Integrated_df.columns else None

    known_bacteria = {}      # usual_taxa
    pure_checked = {}        # only 'chk' checked_taxa
    pure_core = {}          # only 'core' core_taxa
    checked_core = {}       # 'chk-core' checked and core taxa
    source ={}
    sources_found = set()
    patterns = ['us', 'core-us', 'chk-us', 'chk-core-us']

    for i, (genus, gid) in enumerate (zip(genera, gids)):
        if source is not None:  # Check if source exists for this genus
            source = str(sources.iloc[i]).strip().lower()
            sources_found.add(source)

            if source in patterns:
                known_bacteria[genus] = int(gid) if str(gid).isdigit() else gid
                continue

            # Then handle other combinations
            if source == 'chk':
                pure_checked[genus] = gid
            elif source == 'core':
                pure_core[genus] = gid
            elif 'chk-core' in source:
                checked_core[genus] = gid

    print("\nDetailed Classification Results:")
    print(f"Known corrosion bacteria: {len(known_bacteria)}")
    print(f"Pure checked bacteria: {len(pure_checked)}")
    print(f"Pure core bacteria: {len(pure_core)}")
    print(f"Checked-core bacteria: {len(checked_core)}")
    print("\nSources found:", sources_found)

    return {
        'known_bacteria': known_bacteria,
        'pure_checked': pure_checked,
        'pure_core': pure_core,
        'checked_core': checked_core
    }

## 2.7. Prepare picrust data and Creating Directories for PICRUSt2 Input
The check_missing_genera function processes the integrated data and handles data quality control. Known problematic genera (e.g., 'Clostridium_sensu_stricto_12', 'Oxalobacteraceae_unclassified') are flagged for exclusion to prevent analysis errors. The function also creates an organized directory structure as outlined in the introduction, with separate paths for different bacterial classifications (known_mic, candidate_mic, etc.) and their respective analysis outputs (EC_predictions, pathway_predictions, KO_predictions). Following function prepares the data for picrust analysis but both dataframes the abundance.biom and Integrated have some bacteria that were no sequenciated mostly cause are no known specimens. So it is necesary to do same procedure to both dfs.

In [ ]:
def prepare_picrust_data(Integrated_df, aligned_file, function_type='simple'):
    """
    Prepare data for PICRUSt analysis with choice of  function_type method

    Args:
        Integrated_df: Input DataFrame
        aligned_file: Path to aligned sequences
        function_type: 'simple' or 'detailed'
    """
    # Get bacteria source_groups based on chosen  function_type
    if  function_type == 'simple':
        source_groups = get_bacteria_sources_simple(Integrated_df)
    else:
        source_groups= get_bacteria_sources_detailed(Integrated_df)

    # Create appropriate directory structure
    create_directory_structure(function_type)

    return source_groups

def create_directory_structure(function_type='simple'):
    """Create directory structure for PICRUSt analysis"""
    base_dir = Path("/home/beatriz/MIC/2_Micro/data_picrust")
    base_dir.mkdir(parents=True, exist_ok=True)

    if function_type == 'simple':
        directories = SIMPLE_BASE
    else:
        directories = DETAILED_BASE

    # Create all required directories
    for dir_name in directories.values():
        for subdir in SUBDIRS:
            (base_dir / dir_name / subdir).mkdir(parents=True, exist_ok=True)
    logging.info("Directory structure created successfully")
    return True

except Exception as e:
    logging.error(f"Error creating directory structure: {str(e)}")
    return False

In [ ]:
Verify that the files are correct

In [ ]:
def verify_input_files():
    """Verify that input files exist and are readable"""
    missing_files = []
    
    if not fasta_file.exists():
        missing_files.append(str(fasta_file))
    if not biom_table.exists():
        missing_files.append(str(biom_table))
        
    if missing_files:
        logging.error(f"Missing input files: {', '.join(missing_files)}")
        return False
    
    logging.info("All input files found")
    return True

# 3. PICRUSt Pipeline Definition
The pipeline processes the aligned sequence data from notebook 5 that has or not undergo cleaning of the sequences as previously done on section 2. Also processes the biom_table in order to account on this anylsis on abundance. It queries the PICRUSt database to predict potential metabolic pathways for each genus. This prediction is based on evolutionary relationships and known genomic capabilities of related organisms.

In [ ]:
def run_picrust2_pipeline(fasta_file, biom_file, output_dir):
    """
    Run the main PICRUSt2 pipeline on input sequences and BIOM table.

    Args:
        fasta_file: Path to the aligned sequences FASTA file.
        biom_file: Path to the BIOM table (without extra columns).
        output_dir: Directory for PICRUSt2 output.
    """
    try:
        # Run main PICRUSt2 pipeline
        cmd = [
            'picrust2_pipeline.py',
            '-s', fasta_file,        # Input FASTA file with aligned sequences
            '-i', biom_file,         # BIOM table with abundance data
            '-o', output_dir,        # Output directory
            '--processes', '4',      # Parallel processes
            '--verbose',
            '--min_align', '0.25'    # Note the split here
        ]
        subprocess.run(cmd, check=True)

        # Add pathway descriptions if the pathway file exists
        pathway_file = os.path.join(output_dir, 'pathways_out/path_abun_unstrat.tsv.gz')
        if os.path.exists(pathway_file):
            cmd_desc = [
                'add_descriptions.py',
                '-i', pathway_file,
                '-m', 'PATHWAY',
                '-o', os.path.join(output_dir, 'pathways_with_descriptions.tsv')
            ]
            subprocess.run(cmd_desc, check=True)

        print(f"PICRUSt2 pipeline completed successfully for {output_dir}")
        return True

    except subprocess.CalledProcessError as e:
        print(f"Error running PICRUSt2: {e}")
        return False

# 4. Analysis of Pathways
The analysis focuses on metabolic pathways known to be involved in microbially influenced corrosion, including sulfur metabolism, organic acid production, iron metabolism, and biofilm formation. These pathways were selected based on documented mechanisms of known corrosion-inducing bacteria. Separate pipeline runs for simple and detailed classifications ensure proper pathway analysis for each bacterial group.

In [ ]:
def analyze_functional_profiles(picrust_output_dir, bacteria_list):
    """
    Analyze functional profiles with focus on corrosion-relevant pathways

    Parameters:
    picrust_output_dir: directory containing PICRUSt2 output
    bacteria_list: list of bacteria names to analyze
    """
    # Define corrosion-relevant pathways
    relevant_pathways = [
        'Sulfur metabolism',
        'Iron metabolism',
        'Energy metabolism',
        'Biofilm formation',
        'Metal transport',
        'ochre formation',
        'iron oxide deposits',
        'iron precipitation',
        'rust formation',
        'organic acid production',
        'acetate production',
        'lactate metabolism',
        'formate production',
    ]

    try:
        # Read PICRUSt2 output
        pathway_file = os.path.join(picrust_output_dir, 'pathways_with_descriptions.tsv')
        pathways_df = pd.read_csv(pathway_file, sep='\t')

        # Filter for relevant pathways
        filtered_pathways = pathways_df[
            pathways_df['description'].str.contains('|'.join(relevant_pathways),
                                                  case=False,
                                                  na=False)]

        # Calculate pathway abundances per bacteria
        pathway_abundances = filtered_pathways.groupby('description').sum()

        # Calculate pathway similarities between bacteria
        pathway_similarities = {}
        for bacteria in bacteria_list:
            if bacteria in pathways_df.columns:
                similarities = pathways_df[bacteria].corr(pathways_df[list(bacteria_list)])
                pathway_similarities[bacteria] = similarities

        # Predict functional potential
        functional_predictions = {}
        for pathway in relevant_pathways:
            pathway_presence = filtered_pathways[
                filtered_pathways['description'].str.contains(pathway, case=False)
            ]
            if not pathway_presence.empty:
                functional_predictions[pathway] = {
                    'presence': len(pathway_presence),
                    'mean_abundance': pathway_presence.mean().mean(),
                    'max_abundance': pathway_presence.max().max()
                }

        # Calculate correlation scores
        correlation_scores = {}
        for bacteria in bacteria_list:
            if bacteria in pathways_df.columns:
                correlations = pathways_df[bacteria].corr(
                    pathways_df[filtered_pathways.index]
                )
                correlation_scores[bacteria] = {
                    'mean_correlation': correlations.mean(),
                    'max_correlation': correlations.max(),
                    'key_pathways': correlations.nlargest(5).index.tolist()
                }

        comparison_results = {
            'pathway_similarities': pathway_similarities,
            'functional_predictions': functional_predictions,
            'correlation_scores': correlation_scores,
            'pathway_abundances': pathway_abundances.to_dict()
        }

        return filtered_pathways, comparison_results

    except Exception as e:
        print(f"Error in pathway analysis: {str(e)}")
        return None, None

# Testing the pipeline

In [ ]:
# ---- RUNNING THE PIPELINE ----

# Set paths
aligned_fasta_file = Path('/home/beatriz/MIC/2_Micro/data_tree/accession_sequences.fasta') #'data_tree/aligned_sequences_integrate.fasta')
abundance_biom_file =  Path('/home/beatriz/MIC/2_Micro/data_picrust/abundance_accession.biom')
output_dir = 'picrust9_output'

# List of bacteria to analyze
bacteria_of_interest = ['Azospira', 'Brachybacterium', 'Bulleidia']

# Run PICRUSt2
if run_picrust2_pipeline(aligned_fasta_file,
                         abundance_biom_file,
                         output_dir
                        ):
    # Analyze functional profiles if the pipeline completes successfully
    filtered_pathways, abundances = analyze_functional_profiles(output_dir, bacteria_of_interest)

# 5. Functional Analysis
The analysis workflow begins by categorizing bacteria into source groups using the classification functions. These categorized data are then processed through the PICRUSt pipeline to predict metabolic capabilities. The functional analysis examines pathway presence, abundance, and correlations between different bacterial groups to identify potential corrosion-related metabolic patterns.

In [ ]:
def run_functional_analysis(df, Integrated_df, aligned_file, analysis_type='simple'):
    """
    Run complete functional analysis pipeline for either simple or detailed classification

    Parameters:
    df: Input DataFrame
    aligned_file: Path to aligned sequences file
    analysis_type: 'simple' or 'detailed'
    """
    try:
        print(f"\n{'='*50}")
        print(f"Starting {analysis_type} classification analysis")
        print(f"{'='*50}")

        # Prepare data and get source groups
        print("\nStep 1: Preparing data...")

        source_groups = prepare_picrust_data(Integrated_df, aligned_file, function_type=analysis_type)

        if not source_groups:
            raise ValueError("Failed to prepare data: No source groups returned")

        # Base directory for PICRUSt output
        base_dir = Path("/home/beatriz/MIC/2_Micro/data_picrust")

        results = {}

        if analysis_type == 'simple':
            # Run analysis for simple classification
            # Known bacteria
            known_output_dir = base_dir /SIMPLE_BASE['known']
            success_known = run_picrust2_pipeline(aligned_file, df, str(known_output_dir))
            if success_known:
                results_known = analyze_functional_profiles(str(known_output_dir),
                                                        source_groups['known_bacteria'].keys())

            # Other bacteria
            other_output_dir = base_dir / SIMPLE_BASE['other']
            success_other = run_picrust2_pipeline(aligned_file, str(other_output_dir))
            if success_other:
                results_other = analyze_functional_profiles(str(other_output_dir),
                                                        source_groups['other_bacteria'].keys())

        else:
            # Run analysis for detailed classification
            for group, dir_name in DETAILED_BASE.items():

                # Known bacteria
                known_output_dir = base_dir / DETAILED_BASE['known']
                success_known = run_picrust2_pipeline(aligned_file, str(known_output_dir))
                if success_known:
                    results_known = analyze_functional_profiles(str(known_output_dir),
                                                            source_groups['known_bacteria'].keys())

                # Pure checked bacteria
                checked_output_dir = base_dir /  DETAILED_BASE['pure_checked']
                success_checked = run_picrust2_pipeline(aligned_file, str(checked_output_dir))
                if success_checked:
                    results_checked = analyze_functional_profiles(str(checked_output_dir),
                                                            source_groups['pure_checked'].keys())

                # Pure core bacteria
                core_output_dir = base_dir /DETAILED_BASE['pure_core']
                success_core = run_picrust2_pipeline(aligned_file, str(core_output_dir))
                if success_core:
                    results_core = analyze_functional_profiles(str(core_output_dir),
                                                            source_groups['pure_core'].keys())

                # Checked-core bacteria
                checked_core_output_dir = base_dir /DETAILED_BASE['checked_core']
                success_checked_core = run_picrust2_pipeline(aligned_file, str(checked_core_output_dir))
                if success_checked_core:
                    results_checked_core = analyze_functional_profiles(str(checked_core_output_dir),
                                                                    source_groups['checked_core'].keys())
    except subprocess.CalledProcessError as e:
        print(f"Error running PICRUSt2: {e}")

        return "Analysis completed successfully"


diagnostic_optimized_sequences.fasta, picrust_ready_sequences.fasta

In [ ]:
# Run the analysis for both types
# Simple source classification
simple_results = run_functional_analysis(biom_table, aligned_file, analysis_type='simple') # output_biom

# Detailed source classification
detailed_results = run_functional_analysis(biom_table, aligned_file, analysis_type='detailed')

# 6. Findings and Discusion

In [ ]:
def run_picrust2_pipeline(fasta_file, output_dir, min_align =0.5):
    """
    Run PICRUSt2 pipeline with improved error handling and path management

    Args:
        fasta_file: Path to aligned sequences fasta file (str or Path)
        output_dir: Directory for PICRUSt2 output (str or Path)
    """
    import subprocess
    import os
    from pathlib import Path

    # Convert paths to strings
    fasta_file = str(fasta_file)
    output_dir = str(output_dir)

    try:
        # Verify picrust2 is available
        picrust_check = subprocess.run(['which', 'picrust2_pipeline.py'],
                                     capture_output=True,
                                     text=True)
        if picrust_check.returncode != 0:
            raise RuntimeError("picrust2_pipeline.py not found. Please ensure PICRUSt2 is properly installed.")

        # Create output directory
        os.makedirs(output_dir, exist_ok=True)

        # Construct command as a single string
        cmd = f"picrust2_pipeline.py -s {fasta_file} -i {fasta_file} -o {output_dir} --processes 1 --verbose"

        # Run pipeline
        print(f"Running command: {cmd}")
        process = subprocess.run(cmd,
                               shell=True,  # Use shell to handle command string
                               check=True,
                               capture_output=True,
                               text=True)

        print("PICRUSt2 Output:")
        print(process.stdout)

        if process.stderr:
            print("Warnings/Errors:")
            print(process.stderr)

        # Add descriptions if pathway file exists
        pathway_file = os.path.join(output_dir, 'pathways_out/path_abun_unstrat.tsv.gz')
        if os.path.exists(pathway_file):
            desc_cmd = f"add_descriptions.py -i {pathway_file} -m PATHWAY -o {os.path.join(output_dir, 'pathways_with_descriptions.tsv')}"
            subprocess.run(desc_cmd, shell=True, check=True)

        print(f"PICRUSt2 pipeline completed successfully for {output_dir}")
        return True

    except subprocess.CalledProcessError as e:
        print(f"Error running PICRUSt2 command: {e}")
        print(f"Command output: {e.output}")
        return False
    except Exception as e:
        print(f"Error in pipeline: {str(e)}")
        return False

In [ ]:
# For original sequences
aligned_file = Path("/home/beatriz/MIC/2_Micro/data_tree/aligned_sequences_integrate.fasta")
output_dir = Path("/home/beatriz/MIC/2_Micro/data_picrust/original_results")
success = run_picrust2_pipeline(aligned_file, output_dir)

# For improved sequences
optimized_file = Path("/home/beatriz/MIC/2_Micro/data_tree/picrust_optimized_sequences.fasta")
optimized_output = Path("/home/beatriz/MIC/2_Micro/data_picrust/optimized_results")
success_opt = run_picrust2_pipeline(optimized_file, optimized_output)